HW2 testing bench  

In [3]:
import numpy as np
import heapq as h
import seaborn as sns

In [28]:
class Function_node:
    def __init__(self, function_name, children=None, value=None):
        self.function_name = function_name
        self.children = children if children else []
        self.value = value
        
    def __str__(self): #print statement
        if  self.function_name in ['var','const']:
            return f"{self.value}"
        else:
            return f"{self.function_name}"

'''
    def evaluate(self, x = None): #shall return the constant value of its children. If impossible you should evaluate children first. 
        if self.value is not None:
            return self.value
        if self.function_name == '+':
            return self.children[0].evaluate(variables) + self.children[1].evaluate(variables)
        elif self.function_name == '*':
            return self.children[0].evaluate(variables) * self.children[1].evaluate(variables)
        elif self.function_name == '/':
            return self.children[0].evaluate(variables) * self.children[1].evaluate(variables)
        elif self.function_name == 'cos':
            return np.cos(self.children[0].evaluate(variables))
        elif self.function_name == 'sin':
            return np.sin(self.children[0].evaluate(variables))
        elif self.function_name == 'var':
            return x
        elif self.function_name == 'const':
            return self.value
'''

In [5]:
# https://github.com/swacad/numpy-heap/blob/master/heap.py

class NP_Heap(Function_node): 
    def __init__(self,length = 2): 
        self.heap   = np.full(length,np.nan)
        

    def get_parent_idx(self, child_idx):
        if child_idx % 2 == 0:
            parent_idx = int(child_idx / 2 - 1)
        else:
            parent_idx = int(np.floor(child_idx / 2))
        return parent_idx

    def get_parent(self, child_idx):
        parent_idx = self.get_parent_idx(child_idx)
        return self.heap[parent_idx] 
    
    def get_left_child_idx(self, parent_idx):
        return 2 * parent_idx + 1

    def get_right_child_idx(self, parent_idx):
        return 2 * parent_idx + 2

    def get_left_child(self, parent_idx):
        left_child = self.heap[self.get_left_child_idx(parent_idx)]
        return left_child

    def get_right_child(self, parent_idx):
        right_child = self.heap[self.get_right_child_idx(parent_idx)]
        return right_child

    def get_children_idx(self, parent_idx):
        return 2 * parent_idx + 1, 2 * parent_idx + 2

    def get_children(self, parent_idx):
        child_1_idx, child_2_idx = self.get_parent_idx(parent_idx)
        return self.heap[child_1_idx], self.heap[child_2_idx]
    
    def print_arr(self):
        print(np.stack((np.arange(self.heap.size),self.heap)))
        return None
    
    
    
    def insert(self,parent_indx, value):
        # check for size availibility or resize
        # insert child of parent if 
        
        #check heap length to add to parent:
        while self.heap.size < self.get_right_child_idx(parent_indx):
            #print('doubled',self.heap.size ,self.get_right_child_idx(parent_indx)  )
            self.heap = np.append(self.heap, np.full(self.heap.size, np.nan)) #doubles array in size with a string of
            
                
        L,R = self.get_children_idx(parent_indx)

        if np.isnan(self.heap[L]): 
            self.heap[L] = value
        
        elif np.isnan(self.heap[R]): 
            self.heap[R] = value
            
        else: #insert to the left child. 
            print('Parent children filled') #recursive and insert down?
            #self.insert(self,parent_indx = L, value = value )
        
        return None


In [30]:
ww = Function_node('mult')
children = [2,3]
ww.children = children

rslt = ww.evaluate()
print(rslt)


None


In [55]:
hh = NP_Heap(length = 2)
hh.heap[0] = Function_node('mult')
hh.insert(parent_indx = 0,value = Function_node('mult'))
hh.insert(parent_indx = 1,value = Function_node('mult'))

hh.print_arr()


TypeError: float() argument must be a string or a real number, not 'Function_node'

In [24]:
a2  = np.full(3,np.nan).astype(Function_node)
nde = Function_node('mult')
print(a2, type(nde))
a2[0] = Function_node('mult')
print(a2[0])

[nan nan nan] <class '__main__.Function_node'>
function name mult
mult
